In [11]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime
import warnings
warnings.filterwarnings('ignore')
import re
import pandas as pd
import time
from datetime import datetime

In [12]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [13]:
news_df = pd.DataFrame(
    data=None,
    index=None,
    columns=['date','title','link','content']
)

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

중복 제거 후 행 개수:  0


In [14]:
# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    res = requests.get(url, headers = headers, verify=False)
    soup = BeautifulSoup(res.text,'lxml')
    
    return soup

In [15]:
# 뉴스의 기본 정보 가져오기
def get_top3_news_info():
    news_urls =[]
    
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101"
    
    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)
  
    # 해당 분야 상위 뉴스 3개 가져오기

    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        news_url = li.a.attrs.get('href')
        news_urls.append(news_url)

    
    print(news_urls)
    return news_urls

In [16]:
# 상위 3개 뉴스 크롤링
def F_crawling(news_urls):
    news_titles=[]
    news_contents=[]
    news_dates=[]

    for url in news_urls:
        news_html = get_soup_obj(url)


        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")

 
        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        news_date = html_date.attrs['data-date-time']
        news_dates.append(news_date)

    a = pd.DataFrame({'date':news_dates, 'title':news_titles, 'link':news_urls, 'content':news_contents})

    #중복 행 지우기
    a = a.drop_duplicates(keep='first',ignore_index=True)

    return a

In [17]:
# def clearnews():
#     special_char = ' \/:*?\"\"<>|\'\' '
#     for clear in special_char:
#         if clear in title:
#             print(title.find(clear),clear)
#             title = title.replace(clear,'')
#     print(title)

# def clearnews():
#     special_char = ' \/:*?\"\"<>|\'\' '
#     for clear in special_char:
#         if clear in content:
#             print(content.find(clear),clear)
#             content = content.replace(clear,'')
#     print(content)

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    return text  

In [18]:
# while True:
news_urls = get_top3_news_info()
a = F_crawling(news_urls)
news_df=pd.concat([news_df,a])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print(news_df.head())

# time.sleep(50)

['https://n.news.naver.com/mnews/article/003/0011959788?sid=101', 'https://n.news.naver.com/mnews/article/003/0011959787?sid=101', 'https://n.news.naver.com/mnews/article/119/0002728504?sid=101']
                  date                              title   
0  2023-07-07 11:47:05  교원 빨간펜, 7월 한달간 '슈퍼위크'…"할인·사은품 쏜다"  \
1  2023-07-07 11:47:04                        [인사]중소벤처기업부   
2  2023-07-07 11:47:01     킨텍스, 온오프믹스와 함께 "한국판 유레카파크 만든다"   

                                                link   
0  https://n.news.naver.com/mnews/article/003/001...  \
1  https://n.news.naver.com/mnews/article/003/001...   
2  https://n.news.naver.com/mnews/article/119/000...   

                                             content  
0  [\n\n\n\n\n[서울=뉴시스] 교원 빨간펜 '슈퍼위크'. (이미지=교원 제공)...  
1  [\n\t\t\t[서울=뉴시스] ◇과장급 전보▲비상재난담당관 채왕식▶ 네이버에서 뉴...  
2  [\n스타트업 네트워킹과 이벤트플랫폼 활용..스타트업 참여기회 확대\n\n\n\n양...  


In [19]:
NewsDates = news_df['date'].to_list()
NewsTItles = news_df['title'].to_list()
NewsUrls = news_df['link'].to_list()
NewsContents = news_df['content'].to_list()

In [20]:
for date,title,url,content in zip(NewsDates, NewsTItles, NewsUrls, NewsContents):
    sql = "INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES (%s, %s, %s, %s)" % ("'"+date+"'", "'"+title+"'", "'"+url+"'", "'"+content+"'")
    print(sql)
    cur.execute(sql)
conn.commit()
conn.close()

INSERT INTO real_news (NewsDates,NewsTitles,NewsUrls,NewsContents) VALUES ('2023-07-07 11:47:05', '교원 빨간펜, 7월 한달간 '슈퍼위크'…"할인·사은품 쏜다"', 'https://n.news.naver.com/mnews/article/003/0011959788?sid=101', '[




[서울=뉴시스] 교원 빨간펜 '슈퍼위크'. (이미지=교원 제공) 2023.07.07. photo@newsis.com[서울=뉴시스] 배민욱 기자 = 교원 빨간펜이 여름방학을 맞아 7월 한달간 할인 이벤트를 진행한다.7일 교원에 따르면 빨간펜 슈퍼위크 이벤트는 기존·신규 고객을 대상으로 전집, 아이캔두 상품을 할인해 선보인다. 교원 빨간펜은 10일까지 전집을 구매하는 고객을 대상으로 전집 전 품목을 10% 할인된 가격에 제공한다. 또 전집을 묶음 구매하는 고객에게는 어린이용 독서대와 독서등을 증정한다.13일까지 아이캔두를 구매하는 전 고객은 10% 할인 혜택을 받는다. 교원 빨간펜은 ▲아이캔두 초등 ▲솔루토이 국어 ▲솔루토이 한자로 구성된 독서 패키지 구매 시 '국어의 정석'을 특별 사은품으로 제공한다.7월 한달간 아이캔두와 도요새를 신규로 가입하는 고객은 신규 패드 렌탈 시 렌탈료 50% 할인, 리퍼 패드 2년 무상 렌탈, 패드 구매 시 20만원 할인 혜택을 얻는다.
]')


ProgrammingError: (1064, 'You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near \'슈퍼위크\'…"할인·사은품 쏜다"\', \'https://n.news.naver.com/mnews/...\' at line 1')

In [ ]:
# ###데이터 프레임으로 만들기###
# from datetime import date, timedelta
# import pandas as pd

# #데이터 프레임 만들기

# # list=[]
# a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':news_urls,'content':news_contents})

# # list.append([news_df,a])

# #데이터 프레임 저장
# now = datetime.now() 
# a.to_csv('C:\\big16\\trading_project\\ssuyan\\REAL_NEWS\\_{}.csv'.format(now.strftime('%Y%m%d_%H%M%S')),encoding='utf-8-sig',index=False)
